
# 1_noaa_import.ipynb

This file imports the raw NOAA temperature files (average, min, and max temperature for each month for each state), reshapes them, and creates the 5 main variables of interest:
- `pastyear`
- `year1`
- `year2`
- `year3`
- `year4`

In [1]:
# Libraries
import pandas as pd
import numpy as np

In [2]:
# Create month abbreviation to numeric number crosswalk
month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_num = list(range(1,13))
months = pd.DataFrame({'month': month, 'month_num': month_num})

In [3]:
# Import fips code list
fips_codes = pd.read_csv(os.getcwd() + '/data/state_fips.txt', sep = '\t')
fips_codes['polyname'] = fips_codes['polyname'].str.lower()

In [4]:
# Import NOAA geography code list
noaa_codes_st = pd.read_csv(os.getcwd() + '/data/noaa_state_codes_st.csv')
noaa_codes_st['polyname'] = noaa_codes_st['noaa_geoname'].str.lower()
noaa_codes_st = noaa_codes_st[noaa_codes_st['noaa_geocode'] < 100]
noaa_codes_st = noaa_codes_st.merge(fips_codes, on = 'polyname', how = 'left')
noaa_codes_st = noaa_codes_st[['noaa_geocode', 'polyname', 'state_fips']]



In [5]:
# Read in NOAA temperature fixed width files
avgtemp_raw = pd.read_fwf(os.getcwd() + '/data/climdiv-tmpcst-v1.0.0-20200904',
                          names = ['noaa_geocode','division', 'element', 'year','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                          widths = [3, 1, 2, 4, 7, 7 , 7, 7, 7, 7, 7, 7, 7, 7, 7, 7 ]).drop(columns = ['division', 'element'])
avgtemp_raw = avgtemp_raw[avgtemp_raw['noaa_geocode'] < 100]

mintemp_raw = pd.read_fwf(os.getcwd() + '/data/climdiv-tminst-v1.0.0-20200904',
                          names = ['noaa_geocode','division', 'element', 'year','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                          widths = [3, 1, 2, 4, 7, 7 , 7, 7, 7, 7, 7, 7, 7, 7, 7, 7 ]).drop(columns = ['division', 'element'])
mintemp_raw = mintemp_raw[mintemp_raw['noaa_geocode'] < 100]

maxtemp_raw = pd.read_fwf(os.getcwd() + '/data/climdiv-tmaxst-v1.0.0-20200904',
                          names = ['noaa_geocode','division', 'element', 'year','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                          widths = [3, 1, 2, 4, 7, 7 , 7, 7, 7, 7, 7, 7, 7, 7, 7, 7 ]).drop(columns = ['division', 'element'])
maxtemp_raw = maxtemp_raw[maxtemp_raw['noaa_geocode'] < 100]

avgtemp_raw.head()

,noaa_geocode,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1,1895,43.1,37.4,54.5,63.4,69.5,77.5,79.2,79.5,77.8,59.7,53.2,44.9
1,1,1896,43.5,47.7,52.5,68.0,75.9,77.4,81.2,82.2,75.9,63.2,57.3,46.4
2,1,1897,41.8,51.1,60.2,62.4,69.0,81.2,81.5,78.8,75.6,67.1,54.2,47.4
3,1,1898,49.0,46.1,59.2,58.8,74.1,80.4,80.0,78.8,75.2,61.0,49.8,43.4
4,1,1899,43.8,40.0,55.6,61.7,76.1,79.9,80.4,80.9,72.8,66.0,55.3,44.7


In [6]:
# Reshape each NOAA temperature dataset from wide to long
avgtemp_long_a = (avgtemp_raw.melt(id_vars=['noaa_geocode', 'year'], value_vars = month)
                             .rename(columns = {'variable':'month', 'value':'avg_temp'})
                             .merge(months, on = 'month', how = 'left'))

mintemp_long_a = (mintemp_raw.melt(id_vars=['noaa_geocode', 'year'], value_vars = month)
                             .rename(columns = {'variable':'month', 'value':'min_temp'})
                             .merge(months, on = 'month', how = 'left'))

maxtemp_long_a = (maxtemp_raw.melt(id_vars=['noaa_geocode', 'year'], value_vars = month)
                             .rename(columns = {'variable':'month', 'value':'max_temp'})
                             .merge(months, on = 'month', how = 'left'))

alltemp_long_a = avgtemp_long_a.merge(mintemp_long_a, on = ['noaa_geocode', 'month', 'month_num', 'year'], how = 'left')
alltemp_long_b = alltemp_long_a.merge(maxtemp_long_a, on = ['noaa_geocode', 'month', 'month_num', 'year'], how = 'left')
df_w_measures = alltemp_long_a.sort_values(['noaa_geocode', 'year', 'month_num'])
df_w_measures.head()

,noaa_geocode,year,month,avg_temp,month_num,min_temp
0,1,1895,jan,43.1,1,33.4
6144,1,1895,feb,37.4,2,26.8
12288,1,1895,mar,54.5,3,42.4
18432,1,1895,apr,63.4,4,51.2
24576,1,1895,may,69.5,5,58.4


In [7]:
# Measure Creator: creates temperature variables for analysis
def measure_creator(measure, measure_name):

    df_w_measures = alltemp_long_b.copy().sort_values(['noaa_geocode', 'year', 'month_num'])

    df_w_measures["lead2"] = df_w_measures.groupby('noaa_geocode')[measure].shift(-2)
    df_w_measures["lead1"] = df_w_measures.groupby('noaa_geocode')[measure].shift(-1)
    df_w_measures["lag0"] = df_w_measures.groupby('noaa_geocode')[measure].shift(0)
    df_w_measures["lag1"] = df_w_measures.groupby('noaa_geocode')[measure].shift(1)
    df_w_measures["lag2"] = df_w_measures.groupby('noaa_geocode')[measure].shift(2)
    df_w_measures["lag3"] = df_w_measures.groupby('noaa_geocode')[measure].shift(3)
    df_w_measures["lag4"] = df_w_measures.groupby('noaa_geocode')[measure].shift(4)
    df_w_measures["lag5"] = df_w_measures.groupby('noaa_geocode')[measure].shift(5)
    df_w_measures["lag6"] = df_w_measures.groupby('noaa_geocode')[measure].shift(6)
    df_w_measures["lag7"] = df_w_measures.groupby('noaa_geocode')[measure].shift(7)
    df_w_measures["lag8"] = df_w_measures.groupby('noaa_geocode')[measure].shift(8)
    df_w_measures["lag9"] = df_w_measures.groupby('noaa_geocode')[measure].shift(9)
    df_w_measures["lag10"] = df_w_measures.groupby('noaa_geocode')[measure].shift(10)
    df_w_measures["lag11"] = df_w_measures.groupby('noaa_geocode')[measure].shift(11)
    df_w_measures["lag12"] = df_w_measures.groupby('noaa_geocode')[measure].shift(12)
    df_w_measures["lag24"] = df_w_measures.groupby('noaa_geocode')[measure].shift(24)
    df_w_measures["lag36"] = df_w_measures.groupby('noaa_geocode')[measure].shift(36)
    df_w_measures["lag48"] = df_w_measures.groupby('noaa_geocode')[measure].shift(48)
    df_w_measures["lag60"] = df_w_measures.groupby('noaa_geocode')[measure].shift(60)

    df_w_measures['first3months'] = df_w_measures[['lead2', 'lead1', 'lag0']].mean(axis=1)
    df_w_measures['trimester3'] = df_w_measures[['lag1', 'lag2', 'lag3']].mean(axis=1)
    df_w_measures['trimester2'] = df_w_measures[['lag4', 'lag5', 'lag6']].mean(axis=1)
    df_w_measures['trimester1'] = df_w_measures[['lag7', 'lag8', 'lag9']].mean(axis=1)
    df_w_measures['ninemonth'] = df_w_measures[['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9']].mean(axis=1)
    df_w_measures['pastyear'] = df_w_measures[['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11', 'lag12']].mean(axis=1)

    df_w_measures["year1"] = df_w_measures.groupby('noaa_geocode')['pastyear'].shift(-12)
    df_w_measures["year2"] = df_w_measures.groupby('noaa_geocode')['pastyear'].shift(-24)
    df_w_measures["year3"] = df_w_measures.groupby('noaa_geocode')['pastyear'].shift(-36)
    df_w_measures["year4"] = df_w_measures.groupby('noaa_geocode')['pastyear'].shift(-48)
    df_w_measures["year5"] = df_w_measures.groupby('noaa_geocode')['pastyear'].shift(-60)

    df_w_measures['pastyearx'] = df_w_measures[['lag0', 'lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11']].mean(axis=1)

    df_w_measures["year1x"] = df_w_measures.groupby('noaa_geocode')['pastyearx'].shift(-12)
    df_w_measures["year2x"] = df_w_measures.groupby('noaa_geocode')['pastyearx'].shift(-24)
    df_w_measures["year3x"] = df_w_measures.groupby('noaa_geocode')['pastyearx'].shift(-36)
    df_w_measures["year4x"] = df_w_measures.groupby('noaa_geocode')['pastyearx'].shift(-48)
    df_w_measures["year5x"] = df_w_measures.groupby('noaa_geocode')['pastyearx'].shift(-60)



    keep_same = {'noaa_geocode', 'year', 'month', 'month_num', 'avg_temp', 'min_temp', 'max_temp'}
    df_w_measures.columns = ['{}{}'.format(c, '' if c in keep_same else "_" + measure_name)
               for c in df_w_measures.columns]

    return df_w_measures

measures_max = measure_creator('max_temp', "MAX").copy()
measures_avg = measure_creator('avg_temp', "AVG").copy()
measures_min = measure_creator('min_temp', "MIN").copy()

measures_avg.head()

,noaa_geocode,year,month,avg_temp,month_num,min_temp,max_temp,lead2_AVG,lead1_AVG,lag0_AVG,...,year2_AVG,year3_AVG,year4_AVG,year5_AVG,pastyearx_AVG,year1x_AVG,year2x_AVG,year3x_AVG,year4x_AVG,year5x_AVG
0,1,1895,jan,43.1,1,33.4,52.7,54.5,37.4,43.1,...,64.266667,64.191667,62.983333,63.100000,43.10,61.675000,64.125000,64.791667,62.550000,63.083333
6144,1,1895,feb,37.4,2,26.8,48.1,63.4,54.5,37.4,...,64.125000,64.791667,62.550000,63.083333,40.25,62.533333,64.408333,64.375000,62.041667,63.450000
12288,1,1895,mar,54.5,3,42.4,66.5,69.5,63.4,54.5,...,64.408333,64.375000,62.041667,63.450000,45.00,62.366667,65.050000,64.291667,61.741667,63.225000
18432,1,1895,apr,63.4,4,51.2,75.7,77.5,69.5,63.4,...,65.050000,64.291667,61.741667,63.225000,49.60,62.750000,64.583333,63.991667,61.983333,63.433333
24576,1,1895,may,69.5,5,58.4,80.6,79.2,77.5,69.5,...,64.583333,63.991667,61.983333,63.433333,53.58,63.283333,64.008333,64.416667,62.150000,63.041667


In [8]:
# Combine all measures_ datasets
alltemp_long_c = measures_avg.merge(measures_min, on = ['noaa_geocode', 'year', 'month', 'month_num', 'avg_temp', 'min_temp', 'max_temp'], how = 'left')
alltemp_long_c = alltemp_long_c.merge(measures_max, on = ['noaa_geocode', 'year', 'month', 'month_num', 'avg_temp', 'min_temp', 'max_temp'], how = 'left')

# Drop lag and lead variables to simplify the DataFrame
alltemp_long_d = alltemp_long_c[alltemp_long_c.columns.drop(list(alltemp_long_c.filter(regex='lag')))]
alltemp_long_d = alltemp_long_d[alltemp_long_d.columns.drop(list(alltemp_long_d.filter(regex='lead')))]


In [9]:
alltemp_long_d.head()

,noaa_geocode,year,month,avg_temp,month_num,min_temp,max_temp,first3months_AVG,trimester3_AVG,trimester2_AVG,...,year2_MAX,year3_MAX,year4_MAX,year5_MAX,pastyearx_MAX,year1x_MAX,year2x_MAX,year3x_MAX,year4x_MAX,year5x_MAX
0,1,1895,jan,43.1,1,33.4,52.7,45.000000,NaN,NaN,...,75.783333,75.925000,74.216667,74.900000,52.700000,73.091667,75.708333,76.508333,73.825000,74.975000
1,1,1895,feb,37.4,2,26.8,48.1,51.766667,43.100000,NaN,...,75.708333,76.508333,73.825000,74.975000,50.400000,74.000000,75.875000,76.258333,73.275000,75.391667
2,1,1895,mar,54.5,3,42.4,66.5,62.466667,40.250000,NaN,...,75.875000,76.258333,73.275000,75.391667,55.766667,73.783333,76.375000,76.258333,73.100000,75.116667
3,1,1895,apr,63.4,4,51.2,75.7,70.133333,45.000000,NaN,...,76.375000,76.258333,73.100000,75.116667,60.750000,74.191667,75.891667,75.941667,73.233333,75.458333
4,1,1895,may,69.5,5,58.4,80.6,75.400000,51.766667,43.1,...,75.891667,75.941667,73.233333,75.458333,64.720000,74.791667,75.425000,76.425000,73.283333,75.108333


In [10]:
alltemp_long_e = (alltemp_long_d.merge(noaa_codes_st, on = ['noaa_geocode'], how="left")
                                .rename(columns = {'month_num':'bmonth',
                                                   'month':'bmonth_name',
                                                   'year':'byear',
                                                   'polyname':'bpl_name',
                                                   'state_fips': 'BPL_int'}))
alltemp_long_e

,noaa_geocode,byear,bmonth_name,avg_temp,bmonth,min_temp,max_temp,first3months_AVG,trimester3_AVG,trimester2_AVG,...,year4_MAX,year5_MAX,pastyearx_MAX,year1x_MAX,year2x_MAX,year3x_MAX,year4x_MAX,year5x_MAX,bpl_name,BPL_int
0,1,1895,jan,43.1,1,33.4,52.7,45.000000,NaN,NaN,...,74.216667,74.900000,52.700000,73.091667,75.708333,76.508333,73.825000,74.975000,alabama,1
1,1,1895,feb,37.4,2,26.8,48.1,51.766667,43.100000,NaN,...,73.825000,74.975000,50.400000,74.000000,75.875000,76.258333,73.275000,75.391667,alabama,1
2,1,1895,mar,54.5,3,42.4,66.5,62.466667,40.250000,NaN,...,73.275000,75.391667,55.766667,73.783333,76.375000,76.258333,73.100000,75.116667,alabama,1
3,1,1895,apr,63.4,4,51.2,75.7,70.133333,45.000000,NaN,...,73.100000,75.116667,60.750000,74.191667,75.891667,75.941667,73.233333,75.458333,alabama,1
4,1,1895,may,69.5,5,58.4,80.6,75.400000,51.766667,43.100000,...,73.233333,75.458333,64.720000,74.791667,75.425000,76.425000,73.283333,75.108333,alabama,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73723,50,2020,aug,52.1,8,43.9,60.3,-49.233333,49.066667,13.700000,...,NaN,NaN,35.400000,NaN,NaN,NaN,NaN,NaN,alaska,2
73724,50,2020,sep,-99.9,9,-99.9,-99.9,-99.900000,52.000000,27.633333,...,NaN,NaN,22.800000,NaN,NaN,NaN,NaN,NaN,alaska,2
73725,50,2020,oct,-99.9,10,-99.9,-99.9,-99.900000,1.866667,40.466667,...,NaN,NaN,11.425000,NaN,NaN,NaN,NaN,NaN,alaska,2
73726,50,2020,nov,-99.9,11,-99.9,-99.9,-99.900000,-49.233333,49.066667,...,NaN,NaN,1.008333,NaN,NaN,NaN,NaN,NaN,alaska,2


In [11]:
alltemp_long_e.year1_AVG

0        61.641667
1        61.675000
2        62.533333
3        62.366667
4        62.750000
           ...    
73723          NaN
73724          NaN
73725          NaN
73726          NaN
73727          NaN
Name: year1_AVG, Length: 73728, dtype: float64

In [12]:
measures_avg


,noaa_geocode,year,month,avg_temp,month_num,min_temp,max_temp,lead2_AVG,lead1_AVG,lag0_AVG,...,year2_AVG,year3_AVG,year4_AVG,year5_AVG,pastyearx_AVG,year1x_AVG,year2x_AVG,year3x_AVG,year4x_AVG,year5x_AVG
0,1,1895,jan,43.1,1,33.4,52.7,54.5,37.4,43.1,...,64.266667,64.191667,62.983333,63.100000,43.100000,61.675000,64.125000,64.791667,62.550000,63.083333
6144,1,1895,feb,37.4,2,26.8,48.1,63.4,54.5,37.4,...,64.125000,64.791667,62.550000,63.083333,40.250000,62.533333,64.408333,64.375000,62.041667,63.450000
12288,1,1895,mar,54.5,3,42.4,66.5,69.5,63.4,54.5,...,64.408333,64.375000,62.041667,63.450000,45.000000,62.366667,65.050000,64.291667,61.741667,63.225000
18432,1,1895,apr,63.4,4,51.2,75.7,77.5,69.5,63.4,...,65.050000,64.291667,61.741667,63.225000,49.600000,62.750000,64.583333,63.991667,61.983333,63.433333
24576,1,1895,may,69.5,5,58.4,80.6,79.2,77.5,69.5,...,64.583333,63.991667,61.983333,63.433333,53.580000,63.283333,64.008333,64.416667,62.150000,63.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49151,50,2020,aug,52.1,8,43.9,60.3,-99.9,-99.9,52.1,...,NaN,NaN,NaN,NaN,28.000000,NaN,NaN,NaN,NaN,NaN
55295,50,2020,sep,-99.9,9,-99.9,-99.9,-99.9,-99.9,-99.9,...,NaN,NaN,NaN,NaN,15.975000,NaN,NaN,NaN,NaN,NaN
61439,50,2020,oct,-99.9,10,-99.9,-99.9,-99.9,-99.9,-99.9,...,NaN,NaN,NaN,NaN,5.083333,NaN,NaN,NaN,NaN,NaN
67583,50,2020,nov,-99.9,11,-99.9,-99.9,NaN,-99.9,-99.9,...,NaN,NaN,NaN,NaN,-4.900000,NaN,NaN,NaN,NaN,NaN
